In [1]:
%load_ext autoreload
%autoreload 2

### Importing files

In [2]:
from spectradb.main import Database
from spectradb.dataloaders import FTIRDataLoader, FluorescenceDataLoader
from pathlib import Path
import pandas as pd
import sqlite3
import os

### Locating the data folder and creating the data object

In [3]:
path = Path().resolve().parents[0]/"Only Dataloaders"
data_path_ftir = path/"Example.SPA"
data_path_fl = path/"Example.csv"

In [4]:
ftir = FTIRDataLoader(data_path_ftir)

Data generated from FTIR spectrometer
File: Example


There are two ways to create/access database: 
1) Using context manager
2) Using the standard class

### 1. Using context manager

In [5]:
with Database("db.sqlite") as db: 
    db.add_sample(ftir)

In [6]:
with sqlite3.connect("db.sqlite") as conn:  
    df = pd.read_sql("SELECT * FROM measurements", conn)
conn.close()  # unlike other context managers, turns out sqlite requires manual closing
df

,measurement_id,sample_id,instrument_id,measurement_date,sample_name,internal_code,collected_by,comments,data,signal_metadata,date_added
0,1,FTIR_1,FTIR,2024-03-24,NA,NA,NA,NA,"[3.7390708923339844, 3.7333016395568848, 3.686...","{""Wavenumbers"": [3999, 3997, 3995, 3993, 3991,...",2024-09-15 13:07:12


### 2. Using class

In [7]:
os.remove("db.sqlite")

In [8]:
db = Database("db.sqlite")
db.open_connection()

db.add_sample(ftir)

In [9]:
with sqlite3.connect("db.sqlite") as conn:  
    df = pd.read_sql("SELECT * FROM measurements", conn)
conn.close() 
df

,measurement_id,sample_id,instrument_id,measurement_date,sample_name,internal_code,collected_by,comments,data,signal_metadata,date_added
0,1,FTIR_1,FTIR,2024-03-24,NA,NA,NA,NA,"[3.7390708923339844, 3.7333016395568848, 3.686...","{""Wavenumbers"": [3999, 3997, 3995, 3993, 3991,...",2024-09-15 13:07:12


In [10]:
db.close_connection()

### Trying to add the same sample twice

In [11]:
with Database("db.sqlite") as db: 
    db.add_sample(ftir)

┌───────────────────────────────────────────────┐
│      ❗**Duplicate Entry Detected**❗        │
│                                               │
│ The data you're trying to add already exists. │
│ Check the following for uniqueness:           │
│ • Instrument ID                               │
│ • Sample Name                                 │
│ • Internal Sample Code                        │
│                                               │
│ Please update the information and try again.  │
└───────────────────────────────────────────────┘



### Adding two things at the same time

In [12]:
os.remove("db.sqlite")

In [13]:
fl = FluorescenceDataLoader(data_path_fl)

Data generated from Agilent Cary Eclipse fluorescence spectrometer
File: Example
Samples:
+------------------------------------------------------------+
| Identifier | Sample Name                                   |
+------------------------------------------------------------+
| S1         | 20                                            |
| S2         | 22                                            |
| S3         | 19                                            |
| S4         | 21                                            |
+------------------------------------------------------------+


In [14]:
with Database("db.sqlite") as db: 
    db.add_sample([ftir, fl])

In [15]:
with sqlite3.connect("db.sqlite") as conn:  
    df = pd.read_sql("SELECT * FROM measurements", conn)
conn.close()  
df

,measurement_id,sample_id,instrument_id,measurement_date,sample_name,internal_code,collected_by,comments,data,signal_metadata,date_added
0,1,FTIR_1,FTIR,2024-03-24,NA,NA,NA,NA,"[3.7390708923339844, 3.7333016395568848, 3.686...","{""Wavenumbers"": [3999, 3997, 3995, 3993, 3991,...",2024-09-15 13:07:13
1,2,FL_1,FL,2024-04-29,20,NA,NA,NA,"[[2.9585800170898438, 8.902077674865723, 0.0, ...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2024-09-15 13:07:13
2,3,FL_2,FL,2024-04-29,22,NA,NA,NA,"[[18.348623275756836, 0.0, 0.0, 0.0, 3.0769231...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2024-09-15 13:07:13
3,4,FL_3,FL,2024-04-29,19,NA,NA,NA,"[[2.941176414489746, 2.915452003479004, 0.0, -...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2024-09-15 13:07:13
4,5,FL_4,FL,2024-04-29,21,NA,NA,NA,"[[2.949852466583252, 0.0, 0.0, 3.0395138263702...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2024-09-15 13:07:13
